# Objet : Test pandas

## Objectif

- list vers pandas series

## Résultats
- à préciser

## Usages possibles 
- rempacement _keys et _keys par Series

## Autres points
- à tester

données utilisées : https://files.data.gouv.fr/lcsqa/concentrations-de-polluants-atmospheriques-reglementes/temps-reel/2022/

------
## Initialisation
- lecture des fichiers de 01/2022 issus de l'api (un fichier par jour)

In [1]:
from pprint import pprint
from collections import Counter
from time import time
import csv
import os
#os.chdir('C:/Users/a179227/OneDrive - Alliance/perso Wx/ES standard/python ESstandard/ES')
from util import util
from observation import Ilist, Iindex
from copy import copy

chemin = 'C:/Users/a179227/OneDrive - Alliance/perso Wx/ES standard/Environnemental-Sensing/python/validation/air/data_lcsqa/'

In [4]:
import pandas as pd
data = []
t0 = time()
annee = 2022
mois = 1
jour = 1
for i in range(6):
    file = chemin + 'FR_E2_' + str(annee) + '-' + format(mois, '02d') +'-' + format(jour+i, '02d') +'.csv'
    data.append(pd.read_csv(file, sep=';'))
data2 = pd.concat(data, ignore_index=True, join='inner').astype('category')
data2.pop('valeur brute')
print('data2', len(data2), list(data2), time()-t0)
print(type(data2))


data2 295807 ['Date de début', 'Date de fin', 'Organisme', 'code zas', 'Zas', 'code site', 'nom site', "type d'implantation", 'Polluant', "type d'influence", 'discriminant', 'Réglementaire', "type d'évaluation", 'procédure de mesure', 'type de valeur', 'valeur', 'unité de mesure', 'taux de saisie', 'couverture temporelle', 'couverture de données', 'code qualité', 'validité'] 2.431000232696533
<class 'pandas.core.frame.DataFrame'>


In [6]:
t0=time()
print(data2.__class__.__name__)
idxs2 = Ilist.obj(data2)
print('idxs (len, lenlidx, sumcodec) : ', len(idxs2), len(idxs2.idxlen), sum(idxs2.idxlen), time()-t0)
#idxs2.delindex('valeur brute')
idxs2.setvar('valeur')

DataFrame


ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [ ]:
#calcul de values à partir de keys et codec
t0=time()
for idx in idxs2.lindex:
    val = idx.values
print('list to val old: ', time()-t0)
#génération d'une liste de Series à partir d'une liste de keys
pdkeys=[]
t0=time()
for idx in idxs2.lindex:
    pdkeys.append(pd.Series(idx.keys, dtype='category'))
print('list to cat : ', time()-t0)
#calcul de values à partir de keys et codec avec une Series (default)
t0=time()
for idx in idxs2.lindex:
    val = list(pd.Series(idx.keys, dtype='category').cat.rename_categories(idx.codec))
print('list to val (def): ', time()-t0)
#calcul de values à partir de keys et codec avec une Series (quelconque)
t0=time()
for idx, keys in zip(idxs2.lindex,pdkeys):
    val = list(pd.DataFrame(keys.cat.rename_categories(pd.Series(zip(idx.codec, range(len(idx.codec))))).tolist())[0])
print('list to val (all): ', time()-t0)
#calcul de values à partir d'une Series de keys et codec (default)
t0=time()
for idx, keys in zip(idxs2.lindex,pdkeys):
    val = list(keys.cat.rename_categories(idx.codec))
print('cat to val (def): ', time()-t0)


In [ ]:
import json
t0=time()
full = json.dumps([[name, list(ds)] for name,ds in data2.items()])
print('fullsize', len(full), time()-t0)


In [ ]:
t0=time()
fullsize = len(idxs2.to_obj(encoded=True, modecodec='full'))
print('fullsize', fullsize, time()-t0)
t0=time()
minsize = len(idxs2.to_obj(encoded=True, modecodec='nokeys'))
print('minsize', minsize, time()-t0)

----
## format non optimisé
- le "taux d'unicité" reste à 12% (pas de modification des index)
- le "taux de codage" est de 30% (remplacement des données dupliquées par un entier)
- le gain de taille de fichier par rapport à un fichier "quoté" est de 61%
- l'analyse de la structure montre que les données sont principalement du type "linked" (non ou peu structuré)
- quelques colonnes sont de type "derived". Par exemple les index longitude(43) et latitude(44) sont bien dérivés de l'index coordonneesXY(13)
- le taux de couplage ("linkrate") pour chacun des index est très proche de 0, ce qui signifie que les données devraient être de type "derived" (lien de dépendance par exemple comme entre les trimestres et les mois) ou "coupled" (lien biunivoque).

In [ ]:
t0=time()
defaultsize = len(idxs2.to_obj(encoded=True, modecodec='default'))
print('defaultsize', defaultsize, time()-t0)
print('indicator default : ', idxs2.indicator(fullsize, defaultsize))
pprint(idxs2.indexinfos(keys=['num', 'name', 'lencodec', 'parent', 'typecoupl']), width=120)
pprint(idxs2.indexinfos(keys=['linkrate']))

----
## Format optimisé
- le "taux d'unicité" se dégrade légèrement (passage de 11,6% à 12,1%) par l'ajout d'index supplémentaires
- le "taux de codage" par contre passe de 30% à 16% de par l'optimisation 
- le gain de taille de fichier par rapport à un fichier "quoté" est maintenant de 74%
- l'utilisation d'un format binaire (codage CBOR pour Concise Binary Object Representation RFC 8949) permet d'améliorer encore le gain de taille de fichier (82%)

In [ ]:
#idxs.setcanonorder().sort()
t0=time()
optimizesize = len(idxs2.to_obj(modecodec='optimize', encoded=True))
print('optimizesize ', optimizesize, time()-t0)
print('indicator optimize : ', idxs2.indicator(fullsize, optimizesize))
t0=time()
js = idxs2.to_obj(encoded=True, modecodec='optimize', encode_format='cbor')
cborsize = len(js)
print('cborsize', cborsize, time()-t0)
print('indicator cbor : ', idxs2.indicator(fullsize, cborsize))

----
## Intégrité
- la transformation inverse des données binaires permet de vérifier qu'on retombe bien sur les mêmes données (pas de dégradation)

In [ ]:
t0=time()
idxs3 = Ilist.from_obj(js)
print('fromcbor', len(idxs3), time()-t0)
t0=time()
verif = idxs2 == idxs3
print('controle égalité :', verif, time()-t0)


In [ ]:
import pyarrow
t0=time()
data2.to_parquet('test_parquet', engine='pyarrow')
print('toparquet', time()-t0)
